In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import Image

import os, sys, re, datetime, time
from pathlib import Path
from tqdm import tqdm_notebook

pj_dir = Path(os.getcwd()).parents[0]
data_dir = pj_dir/'data'
img_dir = pj_dir/'images'
src_dir = pj_dir/'src'
sys.path.append(str(src_dir))

from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use("bmh")
import numpy as np
import pandas as pd
import dask.dataframe as dd
from scipy import stats 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import japanize_matplotlib

In [29]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (16, 4)
import logging
logging.basicConfig(level=logging.INFO)

from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [30]:
import pandas as pd

In [31]:
from requests_html import HTMLSession
session = HTMLSession()

In [32]:
endpoint = 'https://rebuild.fm/{}/'

In [37]:
episodes = []
for number in tqdm_notebook(range(1, 223)):
    for plus in ['', 'a']: # Normal / Aftershow
        url = endpoint.format(str(number)+plus)
        r = session.get(url)
        if r.status_code != 200:
            print('error: ', url)
            continue

        date_ = r.html.find('#contents > div > span', first=True).text
        date_ = date_.replace('\n', '-')

        record_time = r.html.find('#contents > div > div.post > p > i', first=True).text
        record_time = record_time.replace('収録時間: ', '').replace(' |', '')

        title = r.html.find('#contents > div > h2 > a', first=True).text

        description = r.html.find('#contents > div > div.post > div.episode-description > p', first=True).text

        persons = []
        persion_elements = r.html.find('#contents > div > div.post > div.episode-description > div.episode-people > ul > li')
        for person_element in persion_elements:
            persons.append(person_element.text)

        shownotes = []
        shownote_elements = r.html.find('#show_notes_ > ul > li > a')
        for shownote_element in shownote_elements:
            shownotes.append(shownote_element.text)

        episode = {
            'date': date_,
            'record_time': record_time, 
            'title': title, 
            'persons': persons,
            'shownotes': shownotes
        }
        episodes.append(episode)
        time.sleep(1)


error:  https://rebuild.fm/1a/
error:  https://rebuild.fm/2a/
error:  https://rebuild.fm/3a/
error:  https://rebuild.fm/4a/
error:  https://rebuild.fm/5a/
error:  https://rebuild.fm/6a/
error:  https://rebuild.fm/7a/
error:  https://rebuild.fm/8a/
error:  https://rebuild.fm/9a/
error:  https://rebuild.fm/10a/
error:  https://rebuild.fm/11a/
error:  https://rebuild.fm/12a/
error:  https://rebuild.fm/13a/
error:  https://rebuild.fm/14a/
error:  https://rebuild.fm/15a/
error:  https://rebuild.fm/16a/
error:  https://rebuild.fm/17a/
error:  https://rebuild.fm/18a/
error:  https://rebuild.fm/19a/
error:  https://rebuild.fm/20a/
error:  https://rebuild.fm/21a/
error:  https://rebuild.fm/22a/
error:  https://rebuild.fm/23a/
error:  https://rebuild.fm/24a/
error:  https://rebuild.fm/25a/
error:  https://rebuild.fm/26a/
error:  https://rebuild.fm/27a/
error:  https://rebuild.fm/28a/
error:  https://rebuild.fm/29a/
error:  https://rebuild.fm/30a/
error:  https://rebuild.fm/31a/
error:  https://

In [38]:
df = pd.DataFrame(episodes)

In [ ]:
df.head(20)

In [ ]:
fig = plt.figure(figsize=(16, 4))
ax = fig.add_subplot(1, 1, 1)
ax.plot([0, 1, 2, 3], [0, 2, 4, 6])

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
fig = plt.figure(figsize=(16, 8))
fig.patch.set_facecolor('white')
ax = df.groupby(pd.Grouper(key='date', freq='1M')).size().plot(linewidth=5, linestyle='--')
ax = df.groupby(pd.Grouper(key='date', freq='1M')).size().rolling(window=4).mean().plot(linewidth=5, ax=ax)
ax.set_title('rebuild 一月あたりの更新回数', fontsize=20)
ax.tick_params(axis='x', labelsize='xx-large')
ax.set_xlabel('')
ax.set_ylabel('')
ax.patch.set_facecolor('white') 

In [8]:
df.groupby(pd.Grouper([key='date', freq='1M'), '']).size().plot()

SyntaxError: invalid syntax (<ipython-input-8-9c79a3d6fed3>, line 1)

In [ ]:
persons = []
for i, row in df.iterrows():
    tmp_persosn = row['persons']
    persons.extend(tmp_persosn)

In [ ]:
s_persons = pd.Series(persons)

In [ ]:
s_persons = s_persons[s_persons != 'miyagawa']

In [ ]:
fig = plt.figure(figsize=(16,8))
fig.patch.set_facecolor('white')

tmp_df = s_persons.value_counts()[:10].to_frame('value').reset_index().rename(columns={'index': 'name'})
ax = sns.barplot(x='value', y='name', data=tmp_df)
max_ = tmp_df['value'].max()

for i, (_, row) in enumerate(tmp_df.iterrows()):
    text = ax.text(row['value'] + max_*.03, i+0.1, row['value'], color='black', ha="center", fontsize=20)

[spine.set_visible(False) for spine in ax.spines.values()]
ax.tick_params(bottom=False, left=False, labelbottom=False)
ax.tick_params(axis='y', labelsize='x-large')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title('rebuild 出演回数ランキング', fontsize=20)
ax.patch.set_facecolor('white') 

ax.patch.set_alpha(0)
plt.grid(False)

In [ ]:
shownotes = []
for i, row in df.iterrows():
    tmp_shownote = row['shownotes']
    shownotes.extend(tmp_shownote)
s_shownotes = pd.Series(shownotes)

In [ ]:
plt.figure(figsize=(16,16))

tmp_df = s_shownotes.value_counts()[:20].to_frame('value').reset_index().rename(columns={'index': 'name'})
ax = sns.barplot(x='value', y='name', data=tmp_df)
max_ = tmp_df['value'].max()

for i, (_, row) in enumerate(tmp_df.iterrows()):
    text = ax.text(row['value'] + max_*.03, i+0.1, row['value'], color='black', ha="center", fontsize=20)

[spine.set_visible(False) for spine in ax.spines.values()]
ax.tick_params(bottom=False, left=False, labelbottom=False)
ax.tick_params(axis='y', labelsize='x-large')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title('shownote ランキング', fontsize=20)

ax.patch.set_alpha(0)
plt.grid(False)

In [ ]:
ignore_word = ['-', '–', 'to', 'for', 'the', 'and', '|', 'in', 'of', 'a', 'is', 'on', 'with', 'how', 'new', 'by', '2', 'at', 'rebuild:']

In [ ]:
shownotes_split = []
for i, row in df.iterrows():
    tmp_shownote = row['shownotes']
    for t in tmp_shownote:
        tmp_split = t.split()
        shownotes_split.extend(tmp_split)
shownotes_split = [s for s in shownotes_split if s.lower() not in ignore_word]
s_shownotes_split = pd.Series(shownotes_split)

In [ ]:
plt.figure(figsize=(16,8))

tmp_df = s_shownotes_split.value_counts()[:10].to_frame('value').reset_index().rename(columns={'index': 'name'})
ax = sns.barplot(x='value', y='name', data=tmp_df)
max_ = tmp_df['value'].max()

for i, (_, row) in enumerate(tmp_df.iterrows()):
    text = ax.text(row['value'] + max_*.03, i+0.1, row['value'], color='black', ha="center", fontsize=20)

[spine.set_visible(False) for spine in ax.spines.values()]
ax.tick_params(bottom=False, left=False, labelbottom=False)
ax.tick_params(axis='y', labelsize='x-large')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title('rebuild 話題ランキング', fontsize=20)

ax.patch.set_alpha(0)
ax.patch.set_facecolor('white') 
plt.grid(False)

In [ ]:
(pj_dir/'data').mkdir(parents=True)

In [39]:
df.to_csv(data_dir/'rebuildfm.csv', index=False)